In [ ]:
!git clone https://github.com/dsa-playground/tvt2023.git
%cd /content/tvt2023/
!git pull
!pip install -r requirements.txt -t "tvt2023"

# Inleiding

### Use case
Titanic algemeen, kans op overleven, maar zou jij het overleefd hebben?!
Met Machine Learning kun je middels algoritmes bepalen of iemand wel of niet overleefd heeft gezien de beschikbare variabelen.  

### Instructies omgeving
Stukje over Google Colab met de belangrijkste toets-combinaties / ... 

### Imports & settings
Importeer de functies benodigd voor de training (of... dit gebeurd in cellen vooraf door cursusleider)

In [ ]:
# Imports
import pandas as pd
from scripts.preprocess.preprocess_frontend import zie_settings, laden_data, opschonen_data, numeriek_maken_data, voeg_passagiers_toe
from scripts.modeling.modeling_frontend import train_and_save_model, voorspelling_genereren

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

In [ ]:
_config = zie_settings()

Toelichting Data Science, train/test principe. 

Laten we eens gaan kijken naar de dataset voor het trainen en testen van een model...


In [ ]:
df_train, df_test = laden_data()
display(df_train.head(), df_test.head())

Nou, daar zit een hoop rommel in, laten we die eerst eens wat begrijpelijker maken:
* uitleg stappen

In [ ]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head(), df_test_clean.head())

Nu we de data geschoond hebben, laten we eens kijken welke inzichten we kunnen halen uit deze data:
* Supervette EDA

In [ ]:
df_train_num, df_test_num = numeriek_maken_data(df_train_clean, df_test_clean)
display(df_train_num.head(), df_test_num.head())

Toelichting dataset (naslagwerk):
...

Laten we onszelf toevoegen aan de dataset.

Mogelijke vraag (om te kijken of ze opletten/begrepen hebben): Welke dataset zou dat zijn?!


In [ ]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_train_extended.tail(), df_test_extended.tail())

In [ ]:
train_and_save_model(df=df_train_extended)

In [ ]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)

In [ ]:
# df_test_extended.tail()
df_train_num.columns

In [ ]:
testlist = df_train_num.columns
m1 = ['Overleefd']
not_in_testlist = list(set(testlist) - set(m1))
not_in_testlist

In [ ]:
from sklearn.model_selection import train_test_split
X=df_train_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
       'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
y=df_train_num['Overleefd']
# X_test = df_test_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
#        'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size = 0.2, random_state=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

acc = []

for i in range(1,20):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train,y_train)
    yhat = knn.predict(X_test)
    acc.append(accuracy_score(y_test,yhat))
    print("For k = ",i," : ",accuracy_score(y_test,yhat))

In [ ]:
# plt.figure(figsize=(8,6))
# plt.plot(range(1,20),acc, marker = "o")
# plt.xlabel("Value of k")
# plt.ylabel("Accuracy Score")
# plt.title("Finding the right k")
# plt.xticks(range(1,20))
# plt.show()
max(acc)

In [ ]:
acc.index(max(acc))

In [ ]:
import plotly.express as px

df = df = pd.DataFrame(dict(
    Num_neighbours = range(1,20),
    accuracy_score = acc
))
fig = px.line(df, x="Num_neighbours", y="accuracy_score")
fig.show()

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 3)
KNN.fit(X,y)
y_pred = KNN.predict(X_test)
df_KNN = pd.DataFrame()
# df_KNN["PassengerId"] = test2["PassengerId"]
# df_KNN["Survived"] = y_pred

In [ ]:
X_test['Overleefd']=y_pred

In [ ]:
X_test

In [ ]:
_config['preprocess']['data']['collect']['transform_multi']

toevoegen_passagiers() # pop-up venster?

EDA

Toelichting trainen / algoritme?
Iets over classificatie algoritme (x aantal opties). Nu zijn immers 2 opties: Wel / niet overleven. 

Willen we nog iets doen met train_test_split? Zo ja, dan test set hernoemen in tekst?

Trainen van model / evaluatie?

Voorspellen testset met deelnemers training + visualisaties